Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [59]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [66]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [67]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [12]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic Regression + L2 Regularization 

The first question is: * where do we add the l2 loss? * 

By analogy with (https://tensorflow.googlesource.com/tensorflow/+/master/tensorflow/models/image/mnist/convolutional.py)

we can do it after the `reduce_mean` step

We can pick the beta by doing multiple runs with beta = 1, 0.1,0.01,... while checking the loss on the validation set 

In [73]:
batch_size = 128
beta_reg = .001 # Loss =  Loss' + beta_reg * ||params||_L2^2

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    X_train = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    Y_train = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    X_valid = tf.constant(valid_dataset)
    X_test = tf.constant(test_dataset)

    # Variables.
    W = tf.Variable(
            tf.truncated_normal([image_size * image_size, num_labels]))
    b = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(X_train, W) + b
    
    loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, Y_train)   )
    l2_reg = beta_reg * (tf.nn.l2_loss(W) + tf.nn.l2_loss(b)) 
    
    loss += l2_reg
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(X_valid, W) + b)
    test_prediction = tf.nn.softmax(tf.matmul(X_test, W) + b)

In [444]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # offset:
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # feed:
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {X_train : batch_data, Y_train: batch_labels}
        
        # run:
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 250 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
               valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6793.472168
Minibatch accuracy: 9.4%
Validation accuracy: 12.5%
Minibatch loss at step 250: 1229.712646
Minibatch accuracy: 55.5%
Validation accuracy: 56.6%


KeyboardInterrupt: 

### now with the Neural Net! 

In [77]:
batch_size = 128

# number of hidden nodes in layer 1  
num_h1 = 1024
# beta regularization: Loss =  Loss' + beta_reg * ||params||_L2^2
beta_reg = .001 

graph = tf.Graph()
with graph.as_default():

    # Input data - Placeholder
    X_train = tf.placeholder(tf.float32,
                         shape=(batch_size, image_size * image_size))
    Y_train = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    X_valid = tf.constant(valid_dataset)
    X_test = tf.constant(test_dataset)

    # Variables - random initialization for weights
    # first matrix N_pixels x N_h1 where N_h1 = number of hidden nodes in firs layer
    
    W = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_h1]))
    b = tf.Variable(tf.zeros([num_h1]))
    # + ReLu
    R  = tf.nn.relu(tf.matmul(X_train, W) + b)
    
    # hidden layer 1 variables 
    H1 = tf.Variable(
    tf.truncated_normal([num_h1, num_labels]))
    b_h1 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(R, H1) + b_h1
    
    # loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, Y_train))
    
    # regularization
    l2_reg = beta_reg * (tf.nn.l2_loss(W) + tf.nn.l2_loss(b) + 
                         tf.nn.l2_loss(H1) + tf.nn.l2_loss(b_h1)) 
    loss += l2_reg
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    # pred = ((Xtest^t * W + b) * H1) + bh1
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(X_valid, W) + b), H1) + b_h1)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(X_test, W) + b), H1) + b_h1)

In [78]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # offset:
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # feed:
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {X_train : batch_data, Y_train: batch_labels}
        
        # run:
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
               valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 647.474121
Minibatch accuracy: 13.3%
Validation accuracy: 18.3%
Minibatch loss at step 500: 196.665894
Minibatch accuracy: 82.8%
Validation accuracy: 60.3%
Minibatch loss at step 1000: 116.655571
Minibatch accuracy: 83.6%
Validation accuracy: 64.3%
Minibatch loss at step 1500: 69.847588
Minibatch accuracy: 86.7%
Validation accuracy: 64.2%
Minibatch loss at step 2000: 41.704449
Minibatch accuracy: 87.5%
Validation accuracy: 65.3%
Minibatch loss at step 2500: 25.713997
Minibatch accuracy: 87.5%
Validation accuracy: 66.6%
Minibatch loss at step 3000: 15.539057
Minibatch accuracy: 89.8%
Validation accuracy: 70.3%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [90]:
subset_train  = train_dataset[:1280,:]
subset_labels = train_labels[:1280,:]
print (subset_train.shape, subset_labels.shape)

(1280, 784) (1280, 10)


In [91]:
batch_size = 128

# number of hidden nodes in layer 1  
num_h1 = 1024
# beta regularization: Loss =  Loss' + beta_reg * ||params||_L2^2
beta_reg = .001 

graph = tf.Graph()
with graph.as_default():

    # Input data - Placeholder
    X_train = tf.placeholder(tf.float32,
                         shape=(batch_size, image_size * image_size))
    Y_train = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    X_valid = tf.constant(valid_dataset)
    X_test = tf.constant(test_dataset)

    # Variables - random initialization for weights
    # first matrix N_pixels x N_h1 where N_h1 = number of hidden nodes in firs layer
    W = tf.Variable(tf.truncated_normal([image_size * image_size, num_h1]))
    b = tf.Variable(tf.zeros([num_h1]))
    # + ReLu
    R  = tf.nn.relu(tf.matmul(X_train, W) + b)
    
    # hidden layer 1 variables 
    H1 = tf.Variable(tf.truncated_normal([num_h1, num_labels]))
    b_h1 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(R, H1) + b_h1
    
    # loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, Y_train))
    
    # regularization
    l2_reg = beta_reg * (tf.nn.l2_loss(W) + tf.nn.l2_loss(b) + 
                         tf.nn.l2_loss(H1) + tf.nn.l2_loss(b_h1)) 
    loss += l2_reg
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    # pred = ((Xtest^t * W + b) * H1) + bh1
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(X_valid, W) + b), H1) + b_h1)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(X_test, W) + b), H1) + b_h1)

In [92]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # offset:
        offset = (step * batch_size) % (subset_train.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = subset_train[offset:(offset + batch_size), :]
        batch_labels = subset_labels[offset:(offset + batch_size), :]
        
        # feed:
        feed_dict = {X_train : batch_data, Y_train: batch_labels}
        
        # run:
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
               valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 694.216858
Minibatch accuracy: 5.5%
Validation accuracy: 23.8%
Minibatch loss at step 500: 190.374252
Minibatch accuracy: 100.0%
Validation accuracy: 60.0%
Minibatch loss at step 1000: 115.453362
Minibatch accuracy: 100.0%
Validation accuracy: 60.0%
Minibatch loss at step 1500: 70.017273
Minibatch accuracy: 100.0%
Validation accuracy: 60.1%
Minibatch loss at step 2000: 42.462151
Minibatch accuracy: 100.0%
Validation accuracy: 60.3%
Minibatch loss at step 2500: 25.752001
Minibatch accuracy: 100.0%
Validation accuracy: 60.5%
Minibatch loss at step 3000: 15.618672
Minibatch accuracy: 100.0%
Validation accuracy: 60.9%
Test accuracy: 85.3%


Hard to learn with little data

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [93]:
batch_size = 128

# number of hidden nodes in layer 1  
num_h1 = 1024
# beta regularization: Loss =  Loss' + beta_reg * ||params||_L2^2
beta_reg = .001 

graph = tf.Graph()
with graph.as_default():

    # Input data - Placeholder
    X_train = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    Y_train = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    X_valid = tf.constant(valid_dataset)
    X_test = tf.constant(test_dataset)

    # Variables - random initialization for weights
    # first matrix N_pixels x N_h1 where N_h1 = number of hidden nodes in firs layer
    
    W = tf.Variable(tf.truncated_normal([image_size * image_size, num_h1]))
    b = tf.Variable(tf.zeros([num_h1]))
    # + ReLu 
    # + Dropout ( drop 50% of the activations )
    R  = tf.nn.dropout(tf.nn.relu(tf.matmul(X_train, W) + b),  0.5)
    
    # hidden layer 1 variables 
    H1 = tf.Variable(tf.truncated_normal([num_h1, num_labels]))
    b_h1 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(R, H1) + b_h1
    
    # loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, Y_train))
    
    # regularization
    l2_reg = beta_reg * (tf.nn.l2_loss(W) + tf.nn.l2_loss(b) + 
                         tf.nn.l2_loss(H1) + tf.nn.l2_loss(b_h1)) 
    loss += l2_reg
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    # pred = ((Xtest^t * W + b) * H1) + bh1
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(X_valid, W) + b), H1) + b_h1)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(X_test, W) + b), H1) + b_h1)

In [94]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # offset:
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # feed:
        feed_dict = {X_train : batch_data, Y_train: batch_labels}
        
        # run:
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
               valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 770.083923
Minibatch accuracy: 14.1%
Validation accuracy: 22.9%
Minibatch loss at step 500: 217.690826
Minibatch accuracy: 78.1%
Validation accuracy: 62.3%
Minibatch loss at step 1000: 123.139717
Minibatch accuracy: 75.8%
Validation accuracy: 62.1%
Minibatch loss at step 1500: 71.127846
Minibatch accuracy: 79.7%
Validation accuracy: 60.4%
Minibatch loss at step 2000: 42.896503
Minibatch accuracy: 71.9%
Validation accuracy: 65.4%
Minibatch loss at step 2500: 26.325695
Minibatch accuracy: 78.1%
Validation accuracy: 64.1%
Minibatch loss at step 3000: 15.976868
Minibatch accuracy: 85.9%
Validation accuracy: 68.5%
Test accuracy: 91.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Let's try 2 hidden layers!

In [459]:
# params 

num_steps = 5001
print_step = 400
import numpy as np

# beta regularization: Loss =  Loss' + beta_reg * ||params||_L2^2
#beta_reg = 0.0005
#betas = [beta_reg]
betas = [0.001] 

#decay_rate  = 0.9
decay_rates  = [0.95]

# starter  
#starter_rates = [starter_rate]
starter_rates = [0.001,0.00075,0.0005,0.00025,0.0001]
num_tries = 10



In [460]:
# is early stop based on valid accuracy or on Loss?



import itertools
for (beta_reg,decay_rate,starter_rate) in itertools.product(betas,decay_rates,starter_rates):

    batch_size = 64
    # number of hidden nodes in layer 1  
    num_h1 = 1024
    num_h2 = 256
    
    
    # lr decay
    decay_steps = 20000

    # init graph
    graph = tf.Graph()

    with graph.as_default():

        # Input data - Placeholder
        X_train = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
        Y_train = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        X_valid = tf.constant(valid_dataset)
        X_test = tf.constant(test_dataset)

        # Variables - random initialization for weights + ReLu  (drop 50% of the activations)
        H0 = tf.Variable(tf.truncated_normal([image_size * image_size, num_h1]))
        b_h0 = tf.Variable(tf.constant(0.01, shape=[num_h1]))
        R = tf.nn.dropout(tf.nn.relu(tf.matmul(X_train, H0) + b_h0), 0.5)

        # hidden layer 1 variables + ReLu + Dropout (drop 50%)
        H1 = tf.Variable(tf.truncated_normal([num_h1, num_h2]))
        b_h1 = tf.Variable(tf.constant(0.01, shape=[num_h2]))
        R2 = tf.nn.dropout(tf.nn.relu(tf.matmul(R, H1) + b_h1), 0.5)

        # hidden layer 2 variables + logits computation.
        W = tf.Variable(tf.truncated_normal([num_h2, num_labels]))
        b = tf.Variable(tf.constant(0.1, shape=[num_labels]))
        logits = tf.matmul(R2, W) + b

        # loss
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, Y_train))

        # regularization
        l2_reg = beta_reg * (tf.nn.l2_loss(H0) + tf.nn.l2_loss(b_h0) + 
                             tf.nn.l2_loss(H1) + tf.nn.l2_loss(b_h1) + 
                             tf.nn.l2_loss(W) + tf.nn.l2_loss(b)) 
        loss += l2_reg

        # Optimizer.
        global_step = tf.Variable(0.,)
        learning_rate = tf.train.exponential_decay(starter_rate, 
                                                   #global_step,
                                                   global_step * batch_size, 
                                                   decay_steps,   # decay every decay_steps
                                                   decay_rate,     # decay_rate
                                                   staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(
                                learning_rate).minimize(loss, global_step=global_step)

        # Predictions for the training, validation, and test data.
        # pred =   SoftMax (ReLu(ReLu(Xtest^t * H0 + b_0) * H1  + bh1)* W + b )
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax( \
            tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(X_valid, H0) + b_h0), H1) + b_h1), W) + b)
        test_prediction = tf.nn.softmax( \
            tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(X_test, H0) + b_h0), H1) + b_h1), W) + b)

    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("---Initialized ---: beta,d_rate,s_rate= ",beta_reg,decay_rate,starter_rate)
        
        last_validation  = 0.
        
        for step in range(num_steps):
            # offset:
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            # feed:
            feed_dict = {X_train : batch_data, Y_train: batch_labels}

            # run:
            _, l, lr, predictions  = session.run(
              [optimizer, loss, learning_rate, train_prediction], feed_dict=feed_dict)
            if (step % print_step == 0):
                print("Minibatch loss at step %d: %f, lr: %f" % (step, l, lr))
                #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                   valid_prediction.eval(), valid_labels))
                if accuracy(valid_prediction.eval(), valid_labels) > last_validation:
                    #print("decreasing")
                    last_validation = accuracy(valid_prediction.eval(), valid_labels)
                else: 
                    if accuracy(valid_prediction.eval(), valid_labels) < (last_validation / 1.08):
                        print ("early stop at %s steps"%step, "now: %f< before %f\n"%(
                                accuracy(valid_prediction.eval(), valid_labels), last_validation))
                        break
                        
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---Initialized ---: beta,d_rate,s_rate=  0.001 0.95 0.001
Minibatch loss at step 0: 7208.687012, lr: 0.001000
Validation accuracy: 11.4%
Minibatch loss at step 400: 1067.608398, lr: 0.000950
Validation accuracy: 57.4%
Minibatch loss at step 800: 725.521240, lr: 0.000903
Validation accuracy: 57.7%
Minibatch loss at step 1200: 561.579651, lr: 0.000857
Validation accuracy: 56.3%
Minibatch loss at step 1600: 534.363098, lr: 0.000774
Validation accuracy: 56.5%
Minibatch loss at step 2000: 510.598022, lr: 0.000735
Validation accuracy: 56.7%
Minibatch loss at step 2400: 448.126221, lr: 0.000698
Validation accuracy: 55.0%
Minibatch loss at step 2800: 445.365906, lr: 0.000663
Validation accuracy: 52.3%
early stop at 2800 steps now: 52.290000< before 57.700000

Test accuracy: 78.4%
---Initialized ---: beta,d_rate,s_rate=  0.001 0.95 0.00075
Minibatch loss at step 0: 7383.670898, lr: 0.000750
Validation accuracy: 10.8%
Minibatch loss at step 400: 1169.214233, lr: 0.000713
Validation accuracy: 57.